In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

## Langchain Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACKING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
loader

In [4]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsOptimize tracing spend on LangSmithDevelopersHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsAdministratorsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmithRecommended ReadingBefore diving into this content, it might be helpful to read the following:Data Retention Conceptual DocsUsage Limiting Conceptual DocsnoteSome of the features mentioned in this guide are not currently available in Enterprise plan due to its\ncustom nature of 

In [5]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [6]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsOptimize tracing spend on LangSmithDevelopersHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsAdministratorsOptimize tracing spend on LangSmithOn this pageOptimize tracing spe

In [7]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [8]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

APIConnectionError: Connection error.

In [ ]:
vectorstoredb

In [11]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
result[0].page_content

NameError: name 'vectorstoredb' is not defined

In [1]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [2]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000017D4EC293A0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000017D4EC2B140>, root_client=<openai.OpenAI object at 0x0000017D4E774140>, root_async_client=<openai.AsyncOpenAI object at 0x0000017D4EC29400>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr(''))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_ch

In [3]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

APIConnectionError: Connection error.

In [ ]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [ ]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [ ]:
retrieval_chain

In [ ]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

In [ ]:
response

In [ ]:
response['context']